# الفصل 7: بناء تطبيقات الدردشة
## البدء السريع مع واجهة برمجة تطبيقات OpenAI

تم تعديل هذا الدفتر من [مستودع عينات Azure OpenAI](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) الذي يتضمن دفاتر ملاحظات تصل إلى خدمات [Azure OpenAI](notebook-azure-openai.ipynb).

تعمل واجهة برمجة تطبيقات OpenAI في بايثون مع نماذج Azure OpenAI أيضًا، مع بعض التعديلات البسيطة. يمكنك معرفة المزيد عن الفروقات هنا: [كيفية التبديل بين نقاط نهاية OpenAI و Azure OpenAI باستخدام بايثون](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# نظرة عامة  
"النماذج اللغوية الكبيرة هي دوال تقوم بتحويل النص إلى نص. عند إعطائها سلسلة نصية كمدخل، تحاول النماذج اللغوية الكبيرة التنبؤ بالنص الذي سيأتي بعد ذلك"(1). هذا الدليل السريع سيعرّف المستخدمين على مفاهيم أساسية حول النماذج اللغوية الكبيرة، والمتطلبات الأساسية للبدء مع AML، ومقدمة مبسطة حول تصميم التعليمات، بالإضافة إلى عدة أمثلة قصيرة لحالات استخدام مختلفة.


## جدول المحتويات  

[نظرة عامة](../../../../07-building-chat-applications/python)  
[كيفية استخدام خدمة OpenAI](../../../../07-building-chat-applications/python)  
[1. إنشاء خدمة OpenAI الخاصة بك](../../../../07-building-chat-applications/python)  
[2. التثبيت](../../../../07-building-chat-applications/python)    
[3. بيانات الاعتماد](../../../../07-building-chat-applications/python)  

[حالات الاستخدام](../../../../07-building-chat-applications/python)    
[1. تلخيص النص](../../../../07-building-chat-applications/python)  
[2. تصنيف النص](../../../../07-building-chat-applications/python)  
[3. إنشاء أسماء منتجات جديدة](../../../../07-building-chat-applications/python)  
[4. تحسين أداء المصنف](../../../../07-building-chat-applications/python)  

[المراجع](../../../../07-building-chat-applications/python)


### أنشئ أول مطالبة لك  
هذا التمرين القصير سيقدم لك مقدمة أساسية حول كيفية إرسال مطالبات إلى نموذج OpenAI لأداء مهمة بسيطة مثل "التلخيص".

**الخطوات**:  
1. قم بتثبيت مكتبة OpenAI في بيئة بايثون الخاصة بك  
2. حمّل مكتبات المساعدة القياسية وقم بتعيين بيانات الاعتماد الأمنية المعتادة لخدمة OpenAI التي أنشأتها  
3. اختر نموذجاً لمهمتك  
4. أنشئ مطالبة بسيطة للنموذج  
5. أرسل طلبك إلى واجهة برمجة تطبيقات النموذج!


### ١. تثبيت OpenAI


In [ ]:
%pip install openai python-dotenv

### ٢. استيراد مكتبات المساعدة وإنشاء بيانات الاعتماد


In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### ٣. اختيار النموذج المناسب  
نماذج GPT-3.5-turbo أو GPT-4 قادرة على فهم وإنتاج اللغة الطبيعية.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## ٤. تصميم التعليمات

"سر النماذج اللغوية الكبيرة هو أنه من خلال تدريبها على تقليل خطأ التنبؤ عبر كميات هائلة من النصوص، تتعلم هذه النماذج مفاهيم مفيدة لهذه التنبؤات. على سبيل المثال، تتعلم مفاهيم مثل"(١):

* كيفية التهجئة
* كيف تعمل القواعد اللغوية
* كيفية إعادة صياغة الكلام
* كيفية الإجابة على الأسئلة
* كيفية إجراء المحادثات
* كيفية الكتابة بلغات متعددة
* كيفية البرمجة
* وغيرها

#### كيف تتحكم في النموذج اللغوي الكبير
"من بين جميع المدخلات للنموذج اللغوي الكبير، الأكثر تأثيراً هو نص التعليمات (١).

يمكن تحفيز النماذج اللغوية الكبيرة لإنتاج مخرجات بعدة طرق:

التعليمات: أخبر النموذج بما تريد
الإكمال: اجعل النموذج يكمل بداية ما ترغب به
العرض: أرِ النموذج ما تريد، إما من خلال:
بعض الأمثلة في التعليمات
مئات أو آلاف الأمثلة في مجموعة بيانات تدريبية مخصصة"



#### هناك ثلاث إرشادات أساسية لإنشاء التعليمات:

**أرِ ووضّح**. اجعل ما تريده واضحاً سواء عبر التعليمات أو الأمثلة أو الجمع بينهما. إذا كنت تريد من النموذج ترتيب قائمة عناصر أبجدياً أو تصنيف فقرة حسب المشاعر، وضّح له ذلك.

**قدّم بيانات ذات جودة**. إذا كنت تحاول بناء مصنف أو جعل النموذج يتبع نمطاً معيناً، تأكد من وجود أمثلة كافية. راجع أمثلتك جيداً — النموذج غالباً ذكي بما يكفي لتجاوز الأخطاء الإملائية البسيطة ويعطيك رداً، لكنه قد يفترض أن ذلك مقصود ويمكن أن يؤثر على الرد.

**تحقق من الإعدادات.** إعدادات temperature و top_p تتحكم في مدى حتمية النموذج في توليد الرد. إذا كنت تطلب رداً له إجابة واحدة صحيحة فقط، من الأفضل ضبط هذه الإعدادات على قيم منخفضة. إذا كنت تبحث عن ردود متنوعة أكثر، يمكنك رفعها. الخطأ الأكثر شيوعاً هو افتراض أن هذه الإعدادات تتحكم في "الذكاء" أو "الإبداع".

المصدر: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### كرر نفس الطلب، كيف تقارن النتائج؟


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## تلخيص النص  
#### التحدي  
قم بتلخيص النص بإضافة 'tl;dr:' في نهاية المقطع. لاحظ كيف أن النموذج يفهم كيفية تنفيذ عدة مهام دون تعليمات إضافية. يمكنك تجربة مطالبات أكثر وصفية من tl;dr لتغيير سلوك النموذج وتخصيص التلخيص الذي تحصل عليه(3).  

أظهرت الأبحاث الحديثة تقدماً كبيراً في العديد من مهام ومعايير معالجة اللغة الطبيعية من خلال التدريب المسبق على مجموعة ضخمة من النصوص، ثم تحسين النموذج لمهمة محددة. وعلى الرغم من أن هذه الطريقة غالباً ما تكون محايدة من حيث البنية، إلا أنها لا تزال تتطلب مجموعات بيانات مخصصة للتدريب تتكون من آلاف أو عشرات الآلاف من الأمثلة. بالمقابل، يستطيع البشر عادةً أداء مهمة لغوية جديدة من خلال عدد قليل من الأمثلة أو تعليمات بسيطة - وهو أمر لا تزال أنظمة معالجة اللغة الطبيعية الحالية تواجه صعوبة في تحقيقه. هنا نوضح أن زيادة حجم نماذج اللغة يحسن بشكل كبير الأداء المحايد للمهمة مع عدد قليل من الأمثلة، وأحياناً يصل إلى مستوى المنافسة مع أفضل الأساليب السابقة التي تعتمد على تحسين النموذج للمهمة المحددة.



tl;dr


# تمارين لحالات استخدام متعددة  
1. تلخيص النص  
2. تصنيف النص  
3. ابتكار أسماء جديدة للمنتجات


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## تصنيف النص  
#### التحدي  
صنّف العناصر ضمن الفئات التي يتم تزويدها وقت التنفيذ. في المثال التالي، نعرض كلًا من الفئات والنص المطلوب تصنيفه في الموجه (*playground_reference).

استفسار العميل: مرحبًا، أحد أزرار لوحة مفاتيح اللابتوب الخاصة بي انكسر مؤخرًا وأحتاج إلى بديل:

الفئة المصنفة:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## ابتكر أسماء جديدة للمنتجات
#### التحدي
ابتكر أسماء منتجات باستخدام كلمات أمثلة. في هذا التحدي، ندرج في التعليمات معلومات عن المنتج الذي نرغب في ابتكار أسماء له. كما نوفر مثالاً مشابهاً ليوضح النمط الذي نرغب في الحصول عليه. كذلك قمنا بضبط قيمة العشوائية على مستوى عالٍ لزيادة التنوع والحصول على أفكار مبتكرة أكثر.

وصف المنتج: جهاز منزلي لصنع الميلك شيك
كلمات مفتاحية: سريع، صحي، صغير الحجم.
أسماء المنتجات: هوم شيكر، فيت شيكر، كويك شيك، شيك ميكر

وصف المنتج: زوج من الأحذية يمكن أن يناسب أي مقاس قدم.
كلمات مفتاحية: قابل للتكيف، مناسب، أومني-فيت.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# المراجع  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [أمثلة OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [أفضل الممارسات لتخصيص GPT-3 لتصنيف النصوص](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# لمزيد من المساعدة  
[فريق تسويق OpenAI](AzureOpenAITeam@microsoft.com)


# المساهمون
* Louis Li



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للمعلومات الهامة، يُنصح بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسير خاطئ ينشأ عن استخدام هذه الترجمة.
